In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-appetency/sample_submission.csv
/kaggle/input/netflix-appetency/train.csv
/kaggle/input/netflix-appetency/test.csv


In [2]:
data_train = pd.read_csv('/kaggle/input/netflix-appetency/train.csv')
data_test = pd.read_csv('/kaggle/input/netflix-appetency/test.csv')
data_train.head(), data_test.head()

(   id  target feature_0 feature_1 feature_2 feature_3 feature_4  feature_5  \
 0   0       0        C0        C0        C1        C5       C11      37.56   
 1   1       0        C0        C0        C3        C5        C1        NaN   
 2   2       0        C0        C0        C3        C5        C2        NaN   
 3   3       0        C0        C0        C1        C5        C1        NaN   
 4   4       1        C0        C0        C3        C3       C11      37.48   
 
    feature_6  feature_7  ...  feature_497  feature_498  feature_499  \
 0  54.756667  54.756667  ...            0            0            0   
 1        NaN        NaN  ...            0            0            0   
 2        NaN        NaN  ...            0            0            0   
 3        NaN        NaN  ...            0            0            0   
 4  37.480000  37.161333  ...            0            0            0   
 
    feature_500  feature_501  feature_502  feature_503  feature_504  \
 0            0    

In [3]:
combine = [data_train, data_test]
data_train.shape

(70000, 509)

In [4]:
print("Numbers of #1: ", data_train['target'].sum() )
print("Numbers of #0: ", data_train['target'].count() - data_train['target'].sum())
print("Subscription rate: ", data_train['target'].mean()* 100)

Numbers of #1:  20873
Numbers of #0:  49127
Subscription rate:  29.818571428571428


Imbalanced data set(most ppl don't subscribe)
==> base line: **higher than 0.71**

In [5]:
data_train.describe(), data_train.describe(include = ['O']) 

(                 id        target     feature_5     feature_6     feature_7  \
 count  70000.000000  70000.000000  25424.000000  26389.000000  27792.000000   
 mean   50025.281314      0.298186     40.124969     39.975888     38.937569   
 std    28878.675532      0.457465     21.415242     20.591512     20.364727   
 min        0.000000      0.000000   -217.460000    -89.690000    -66.497000   
 25%    24999.750000      0.000000     32.470000     32.470000     31.923333   
 50%    50030.500000      0.000000     37.480000     37.560000     37.030667   
 75%    75027.250000      1.000000     48.062750     48.490000     47.726458   
 max    99999.000000      1.000000    403.880000    639.756667    511.091667   
 
           feature_8     feature_9    feature_10    feature_11    feature_12  \
 count  25424.000000  26389.000000  27792.000000  25424.000000  26389.000000   
 mean      32.120109     31.533970     30.517150      2.891956      2.847829   
 std       13.766195     13.330736    

In [6]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 509 entries, id to feature_506
dtypes: float64(143), int64(274), object(92)
memory usage: 271.8+ MB


# Remving Missing Values

In [7]:
num_na = []
for item in data_train.columns:
    num_na.append(data_train[item].isna().sum())
    
num_na  = np.array(num_na)
num_na  = num_na.reshape(-1, 1)
num_na  = pd.DataFrame(num_na, index = data_train.columns, columns = ['#na'])
num_na['%missing'] = round ((num_na['#na']/ data_train.shape[0] *100), 2)
num_na.sort_values(by = '#na', ascending = False)

,#na,%missing
feature_193,70000,100.00
feature_196,70000,100.00
feature_197,70000,100.00
feature_198,70000,100.00
feature_83,68824,98.32
...,...,...
feature_189,0,0.00
feature_188,0,0.00
feature_187,0,0.00
feature_186,0,0.00


To see how many 'NaN' in each column:
=>Some are even 100% (don't contain any records)
-> can directly drop 

In [8]:
large_na = [item for item in num_na[num_na['%missing'] >= 90.0].index]
for item in combine:
    item.drop(labels = large_na, axis =1, inplace = True)
print(len(large_na))
data_train.shape, data_test.shape

5


((70000, 504), (30000, 503))

Remove cols with null values over 90%

In [9]:
for item in combine:
    for data in item.columns:
        t = item.loc[item[data].notna(), data].head(1)
        if isinstance(t, (int, float)):
            item[data].fillna(item[data].notna().median(), inplace = True)
        else:#object type
            md = item[data].mode()[0]
            item[data].fillna(md, inplace = True)

data_train.isna().sum().mean(), data_test.isna().sum().mean(), 

(0.0, 0.0)

Fill in missing value:
* numeric--> median
* categorical--> freq

In [10]:
#Categorical features name
Categorical_Features = list(data_train.select_dtypes(include = ['object']).columns)
        
#Numeric features name
Numeric_Features = list(data_train.select_dtypes(exclude=['object']).columns[2:]) #skip id,target

# Dealing with Datetime Type Features

In [11]:
import re

date_features = []
for col in Categorical_Features:
    md = data_train[col].mode()[0]
    if re.match('(\d{2,4}(-|\/|\\|\.| )\d{2}(-|\/|\\|\.| )\d{2,4})+', md):
        #2~4 numbers + (-|\/|\\|\.|) + 2 nums +(-|\/|\\|\.|) + 2~4 nums + (-|\/|\\|\.|)
        date_features.append(col)
date_features

['feature_191',
 'feature_192',
 'feature_194',
 'feature_195',
 'feature_199',
 'feature_200',
 'feature_201',
 'feature_202',
 'feature_203',
 'feature_204']

'(\d{2,4}(-|\/|\\|\.| )\d{2}(-|\/|\\|\.| )\d{2,4})+'
\d{2,4}: numbers repeat 2~4 times
(-|\/|\\|\.| ): then followed by '-' or '/' or '\' or '.' or ''
'(...)+': repeats one or more times

In [12]:
for item in combine:
    for col in date_features:
        item[col] = item[col].apply(pd.to_datetime,dayfirst=False)

data_train[date_features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   feature_191  70000 non-null  datetime64[ns]
 1   feature_192  70000 non-null  datetime64[ns]
 2   feature_194  70000 non-null  datetime64[ns]
 3   feature_195  70000 non-null  datetime64[ns]
 4   feature_199  70000 non-null  datetime64[ns]
 5   feature_200  70000 non-null  datetime64[ns]
 6   feature_201  70000 non-null  datetime64[ns]
 7   feature_202  70000 non-null  datetime64[ns]
 8   feature_203  70000 non-null  datetime64[ns]
 9   feature_204  70000 non-null  datetime64[ns]
dtypes: datetime64[ns](10)
memory usage: 5.3 MB


In [13]:
for col in date_features:
    data_train[col+"_year"] = pd.DatetimeIndex(data_train[col]).year
   # train[col+"_month"]=pd.DatetimeIndex(train[col]).month
 
    data_test[col+"_year"] = pd.DatetimeIndex(data_test[col]).year
   # test[col+"_month"]=pd.DatetimeIndex(test[col]).month

data_train

,id,target,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_191_year,feature_192_year,feature_194_year,feature_195_year,feature_199_year,feature_200_year,feature_201_year,feature_202_year,feature_203_year,feature_204_year
0,0,0,C0,C0,C1,C5,C11,37.560,54.756667,54.756667,...,2017,2017,2017,2016,2017,2016,2017,2016,2017,2008
1,1,0,C0,C0,C3,C5,C1,32.470,0.000000,0.000000,...,2015,2015,2016,2016,2017,2015,2015,2016,2017,2015
2,2,0,C0,C0,C3,C5,C2,32.470,0.000000,0.000000,...,2015,2015,2017,2016,2016,2015,2015,2016,2017,2015
3,3,0,C0,C0,C1,C5,C1,32.470,0.000000,0.000000,...,2014,2014,2016,2014,2014,2014,2014,2014,2014,2014
4,4,1,C0,C0,C3,C3,C11,37.480,37.480000,37.161333,...,2009,2013,2017,2016,2013,2016,2013,2016,2013,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99994,0,C0,C0,C3,C1,C11,32.470,0.000000,0.000000,...,2010,2014,2014,2014,2014,2014,2014,2017,2014,2011
69996,99996,0,C0,C0,C5,C5,C2,39.398,35.022000,45.645000,...,2016,2016,2016,2016,2016,2016,2016,2016,2017,2008
69997,99997,0,C0,C0,C3,C2,C11,32.470,0.000000,0.000000,...,2005,2015,2016,2015,2015,2016,2015,2016,2015,2013
69998,99998,0,C0,C0,C1,C5,C1,30.060,35.765000,35.765000,...,2017,2017,2018,2016,2017,2016,2017,2016,2017,2008


In [14]:
for item in combine:
    item.drop(labels = date_features, axis = 1, inplace = True)
print(len(date_features))
data_train.shape, data_test.shape

10


((70000, 504), (30000, 503))

Only keeps the 'year' data 

# Feature Engineering - Numeric Features

In [15]:
import math

std = []
for zstd in Numeric_Features:
    std.append(data_train[zstd].std())
        
std = pd.DataFrame(np.array(std), index = Numeric_Features, columns = ['std'])
z_std = list(std.loc[std['std'] == 0].index)
z_std

['feature_100',
 'feature_149',
 'feature_227',
 'feature_228',
 'feature_252',
 'feature_312',
 'feature_313',
 'feature_357',
 'feature_358',
 'feature_375',
 'feature_377',
 'feature_385',
 'feature_389',
 'feature_395',
 'feature_397',
 'feature_405',
 'feature_409',
 'feature_411',
 'feature_412',
 'feature_413',
 'feature_419',
 'feature_421',
 'feature_425',
 'feature_427',
 'feature_432',
 'feature_433',
 'feature_445',
 'feature_449',
 'feature_451',
 'feature_453',
 'feature_455',
 'feature_459',
 'feature_461',
 'feature_463',
 'feature_465',
 'feature_467',
 'feature_469',
 'feature_472',
 'feature_474',
 'feature_476',
 'feature_478',
 'feature_480',
 'feature_482',
 'feature_484',
 'feature_487',
 'feature_489',
 'feature_491',
 'feature_493',
 'feature_495',
 'feature_498',
 'feature_500',
 'feature_502',
 'feature_505']

In [16]:
for item in combine:
    item.drop(labels = z_std, axis =1, inplace = True)
    
print(len(z_std))
data_train.shape, data_test.shape

53


((70000, 451), (30000, 450))

Remove zero std 

In [17]:
std.drop(labels = z_std, axis = 0, inplace = True)
std.sort_values(by = 'std', ascending = False)

,std
feature_36,9.109474e+08
feature_206,8.700168e+07
feature_207,8.117209e+07
feature_160,7.401401e+07
feature_40,7.290369e+07
...,...
feature_120,1.511696e-02
feature_330,1.463704e-02
feature_85,1.063799e-02
feature_102,9.999571e-03


In [18]:
std['#_nums'] = 0 
for col in std.index:
    std.loc[col, '#_nums'] = data_train[col].nunique()
std

,std,#_nums
feature_5,13.420772,8488
feature_6,23.134012,14004
feature_7,22.969892,18386
feature_8,8.319410,1707
feature_9,8.191971,3939
...,...,...
feature_499,17.344944,2
feature_501,17.344944,2
feature_503,17.344944,2
feature_504,17.344944,2


In [19]:
drop_num = list(std[(std['std'] > 17) & (std['std'] <18) & (std['#_nums'] ==2)].index[1:])
drop_num, len(drop_num)

(['feature_376',
  'feature_384',
  'feature_388',
  'feature_394',
  'feature_396',
  'feature_404',
  'feature_408',
  'feature_410',
  'feature_418',
  'feature_420',
  'feature_424',
  'feature_426',
  'feature_444',
  'feature_448',
  'feature_450',
  'feature_452',
  'feature_454',
  'feature_458',
  'feature_460',
  'feature_462',
  'feature_464',
  'feature_466',
  'feature_468',
  'feature_471',
  'feature_473',
  'feature_475',
  'feature_477',
  'feature_479',
  'feature_481',
  'feature_483',
  'feature_485',
  'feature_486',
  'feature_488',
  'feature_490',
  'feature_492',
  'feature_494',
  'feature_496',
  'feature_497',
  'feature_499',
  'feature_501',
  'feature_503',
  'feature_504',
  'feature_506'],
 43)

去掉 std 一樣的資料 

In [20]:
std[std['#_nums'] <= 100].sort_values(by = '#_nums', ascending = False)

,std,#_nums
feature_343,4.019241,96
feature_56,1.199839,86
feature_170,0.559747,85
feature_74,4.555982,84
feature_361,2.844363,75
...,...,...
feature_384,17.344944,2
feature_138,0.101656,2
feature_376,17.344944,2
feature_374,17.344944,2


類別少於100個的就有233/367 (63%)

In [21]:
bi_cat = std[(std['#_nums'] <= 2) & (std['std'] < 17)].sort_values(by = 'std', ascending = False)
bi_cat

,std,#_nums
feature_261,0.458497,2
feature_307,0.382347,2
feature_51,0.353195,2
feature_263,0.269518,2
feature_136,0.259927,2
feature_329,0.198820,2
feature_267,0.190228,2
feature_373,0.188971,2
feature_126,0.186042,2
feature_332,0.158326,2


std[std['#_cat'] <= 2] ==> 84 cols
len(bi_cat) = 40

In [22]:
bi_cat['sub_rate_0'] = 0
bi_cat['sub_rate_1'] = 0
bi_cat['diff'] = 0
for col in bi_cat.index:
    bi_cat.loc[col, 'sub_rate_0'] = round((data_train[[col, 'target']].groupby([col]).mean().iloc[0, 0]),2)
    bi_cat.loc[col, 'sub_rate_1'] = round((data_train[[col, 'target']].groupby([col]).mean().iloc[1, 0]),2)
    bi_cat['diff'] = abs(bi_cat['sub_rate_0'] - bi_cat['sub_rate_1'])*100
bi_cat.sort_values(by = 'diff', ascending = False)

,std,#_nums,sub_rate_0,sub_rate_1,diff
feature_307,0.382347,2,0.23,0.63,40.0
feature_317,0.005345,2,0.30,0.00,30.0
feature_265,0.067876,2,0.30,0.55,25.0
feature_373,0.188971,2,0.30,0.11,19.0
feature_263,0.269518,2,0.28,0.46,18.0
feature_77,0.104169,2,0.30,0.48,18.0
feature_267,0.190228,2,0.29,0.46,17.0
feature_323,0.021040,2,0.30,0.45,15.0
feature_329,0.198820,2,0.29,0.42,13.0
feature_136,0.259927,2,0.29,0.41,12.0


In [23]:
len(bi_cat.loc[ (bi_cat['std'] < 0.2) & (bi_cat['diff'] < 10) ,('std', 'diff')].index)

27

In [24]:
#Removing std lower than 0.1
for col in std[std['std'] < 0.1].sort_values(by = 'std', ascending = False).index:
    if col not in drop_num:
        drop_num.append(col)
len(drop_num)

86

In [25]:
low_std = std.loc[(std['#_nums']<= 5) & (std['std'] < 0.3)].sort_values(by = ['std', '#_nums'], ascending = False)

low_std['sub_rate_diff'] = 0

for col in low_std.index:
    ma = data_train[[col, 'target']].groupby([col]).mean().max()[0]
    mi = data_train[[col, 'target']].groupby([col]).mean().min()[0]
    low_std.loc[col, 'sub_rate_diff'] = round((ma - mi), 2)
low_std.sort_values(by = 'sub_rate_diff', ascending = False)

,std,#_nums,sub_rate_diff
feature_105,0.062216,3,0.72
feature_314,0.267100,3,0.70
feature_302,0.146798,3,0.41
feature_306,0.133361,3,0.34
feature_308,0.067004,3,0.34
...,...,...,...
feature_116,0.028278,3,0.01
feature_327,0.018124,2,0.01
feature_102,0.010000,2,0.01
feature_113,0.036426,2,0.01


(std['#_cat']<= 5) & (std[0] < 1)==> 107 cols
(std['#_cat']<= 5) & (std[0] < 0.5)==> 91 cols
(std['#_cat']<= 5) & (std[0] < 0.3)==> 79 cols

In [26]:
for col in low_std[low_std['sub_rate_diff'] <= 0.4].index:
    if col not in drop_num:
        drop_num.append(col)

for item in combine:
    item.drop(labels = drop_num, axis = 1, inplace = True)
data_train.shape, data_test.shape

((70000, 329), (30000, 328))

In [27]:
#Get correlations with target (sorted)
corr = data_train.drop(['id', 'target'], axis=1).corrwith(data_train['target']).abs()
corr.head(10).sort_values(ascending = False)

feature_5     0.069287
feature_7     0.067773
feature_6     0.062295
feature_14    0.015659
feature_10    0.008626
feature_13    0.007289
feature_12    0.004518
feature_11    0.001814
feature_8     0.001390
feature_9     0.000900
dtype: float64

In [28]:
corr = pd.DataFrame(corr, columns = ['corr'])
corr['top'] = 0
for col in corr.index:
    corr.loc[col, 'top'] = round((data_train[col].value_counts().max())/70000, 2)
corr

,corr,top
feature_5,0.069287,0.66
feature_6,0.062295,0.65
feature_7,0.067773,0.63
feature_8,0.001390,0.68
feature_9,0.000900,0.67
...,...,...
feature_200_year,0.081535,0.38
feature_201_year,0.083620,0.26
feature_202_year,0.072909,0.61
feature_203_year,0.062113,0.68


In [29]:
drop_corr = []
for col in corr[corr['top'] >= 0.9].index:
    drop_corr.append(col)
for col in corr[corr['corr'] <= 0.003].index:
    if col not in drop_corr:
        drop_corr.append(col)
        
for item in combine:
    item.drop(labels = drop_corr, axis = 1, inplace = True)
data_train.shape, data_test.shape

((70000, 219), (30000, 218))

In [30]:
#data_train['feature_328'].value_counts().head(10), data_train[['feature_328', 'target']].groupby(['feature_328']).mean().sort_values(by = 'target', ascending = False)

In [31]:
Numeric_Features = list(data_train.drop(['id', 'target'], axis = 1).select_dtypes(exclude=['object']).columns) 
data_train[Numeric_Features]

,feature_5,feature_6,feature_7,feature_10,feature_12,feature_13,feature_15,feature_25,feature_26,feature_35,...,feature_191_year,feature_192_year,feature_194_year,feature_195_year,feature_199_year,feature_200_year,feature_201_year,feature_202_year,feature_203_year,feature_204_year
0,37.560,54.756667,54.756667,39.136667,3.026667,3.026667,0.000000,0,0,-17.527805,...,2017,2017,2017,2016,2017,2016,2017,2016,2017,2008
1,32.470,0.000000,0.000000,30.830000,2.500000,2.500000,0.000000,0,0,0.000000,...,2015,2015,2016,2016,2017,2015,2015,2016,2017,2015
2,32.470,0.000000,0.000000,30.830000,2.500000,2.500000,0.000000,0,0,0.000000,...,2015,2015,2017,2016,2016,2015,2015,2016,2017,2015
3,32.470,0.000000,0.000000,30.830000,2.500000,2.500000,0.000000,0,0,0.000000,...,2014,2014,2016,2014,2014,2014,2014,2014,2014,2014
4,37.480,37.480000,37.161333,35.750000,2.500000,2.500000,0.258000,4716,26992,0.000000,...,2009,2013,2017,2016,2013,2016,2013,2016,2013,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,32.470,0.000000,0.000000,30.830000,2.500000,2.500000,0.000000,1647,12835,17.820060,...,2010,2014,2014,2014,2014,2014,2014,2017,2014,2011
69996,39.398,35.022000,45.645000,30.830000,2.500000,2.500000,13.721667,5491,61194,0.000000,...,2016,2016,2016,2016,2016,2016,2016,2016,2017,2008
69997,32.470,0.000000,0.000000,30.830000,2.500000,2.500000,0.000000,19173,57103,0.000000,...,2005,2015,2016,2015,2015,2016,2015,2016,2015,2013
69998,30.060,35.765000,35.765000,35.765000,3.000000,3.000000,0.000000,0,0,-64.459028,...,2017,2017,2018,2016,2017,2016,2017,2016,2017,2008


In [32]:
corr_df = data_train[Numeric_Features].corr()
corr_pairs = corr_df.unstack()
corr_pairs.sort_values()

feature_81        feature_191_year   -0.998106
feature_191_year  feature_81         -0.998106
feature_82        feature_194_year   -0.964569
feature_194_year  feature_82         -0.964569
feature_295       feature_122        -0.837675
                                        ...   
feature_261       feature_261         1.000000
feature_260       feature_260         1.000000
feature_259       feature_259         1.000000
feature_307       feature_307         1.000000
feature_204_year  feature_204_year    1.000000
Length: 18225, dtype: float64

In [33]:
high_corr_pairs = corr_pairs[(abs(corr_pairs) >= 0.9) & (abs(corr_pairs) < 1)]
for f in high_corr_pairs.index:
    f = sorted(f)
high_corr_pairs

feature_6         feature_7      0.975291
feature_7         feature_6      0.975291
feature_12        feature_13     0.925145
feature_13        feature_12     0.925145
feature_25        feature_26     0.908289
                                   ...   
feature_456       feature_440    0.996907
feature_470       feature_43     0.921355
                  feature_143    0.914799
feature_191_year  feature_81    -0.998106
feature_194_year  feature_82    -0.964569
Length: 218, dtype: float64

In [34]:
high_corr = []
i = 0
while i < (len(high_corr_pairs.index)):
    t = high_corr_pairs.index[i]
    if t[0] not in high_corr and t[1] not in high_corr:
        high_corr.append(high_corr_pairs.index[i][0])
    i += 2
len(high_corr),high_corr

(38,
 ['feature_6',
  'feature_12',
  'feature_25',
  'feature_35',
  'feature_43',
  'feature_60',
  'feature_66',
  'feature_82',
  'feature_91',
  'feature_110',
  'feature_128',
  'feature_132',
  'feature_143',
  'feature_154',
  'feature_155',
  'feature_156',
  'feature_157',
  'feature_160',
  'feature_162',
  'feature_166',
  'feature_206',
  'feature_223',
  'feature_256',
  'feature_258',
  'feature_260',
  'feature_353',
  'feature_369',
  'feature_378',
  'feature_380',
  'feature_386',
  'feature_390',
  'feature_400',
  'feature_402',
  'feature_414',
  'feature_416',
  'feature_436',
  'feature_440',
  'feature_191_year'])

In [35]:
for item in combine:
    item.drop(labels = high_corr, axis = 1, inplace = True)
data_train.shape, data_test.shape

((70000, 181), (30000, 180))

In [36]:
data_train

,id,target,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_7,feature_10,...,feature_470,feature_192_year,feature_194_year,feature_195_year,feature_199_year,feature_200_year,feature_201_year,feature_202_year,feature_203_year,feature_204_year
0,0,0,C0,C0,C1,C5,C11,37.560,54.756667,39.136667,...,0,2017,2017,2016,2017,2016,2017,2016,2017,2008
1,1,0,C0,C0,C3,C5,C1,32.470,0.000000,30.830000,...,2,2015,2016,2016,2017,2015,2015,2016,2017,2015
2,2,0,C0,C0,C3,C5,C2,32.470,0.000000,30.830000,...,2,2015,2017,2016,2016,2015,2015,2016,2017,2015
3,3,0,C0,C0,C1,C5,C1,32.470,0.000000,30.830000,...,0,2014,2016,2014,2014,2014,2014,2014,2014,2014
4,4,1,C0,C0,C3,C3,C11,37.480,37.161333,35.750000,...,0,2013,2017,2016,2013,2016,2013,2016,2013,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99994,0,C0,C0,C3,C1,C11,32.470,0.000000,30.830000,...,0,2014,2014,2014,2014,2014,2014,2017,2014,2011
69996,99996,0,C0,C0,C5,C5,C2,39.398,45.645000,30.830000,...,0,2016,2016,2016,2016,2016,2016,2016,2017,2008
69997,99997,0,C0,C0,C3,C2,C11,32.470,0.000000,30.830000,...,1,2015,2016,2015,2015,2016,2015,2016,2015,2013
69998,99998,0,C0,C0,C1,C5,C1,30.060,35.765000,35.765000,...,0,2017,2018,2016,2017,2016,2017,2016,2017,2008


# Numeric Features Normalization

In [37]:
Numeric_Features = list(data_train.drop(['id', 'target'], axis = 1).select_dtypes(exclude=['object']).columns) 

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
st_X = sc.fit_transform(data_train[Numeric_Features])
st_X

array([[ 0.17210126,  1.7108317 ,  0.97703374, ...,  0.3311176 ,
         0.53192374, -0.82435449],
       [-0.20716427, -0.67303048,  0.01439442, ...,  0.3311176 ,
         0.53192374,  1.16086061],
       [-0.20716427, -0.67303048,  0.01439442, ...,  0.3311176 ,
         0.53192374,  1.16086061],
       ...,
       [-0.20716427, -0.67303048,  0.01439442, ...,  0.3311176 ,
        -0.62179872,  0.59365629],
       [-0.38673794,  0.88401884,  0.58629952, ...,  0.3311176 ,
         0.53192374, -0.82435449],
       [-0.20716427, -0.67303048,  0.01439442, ...,  0.3311176 ,
         0.53192374, -0.82435449]])

In [38]:
st_X = pd.DataFrame(st_X, columns = data_train[Numeric_Features].columns)
st_X

,feature_5,feature_7,feature_10,feature_13,feature_15,feature_26,feature_36,feature_37,feature_38,feature_39,...,feature_470,feature_192_year,feature_194_year,feature_195_year,feature_199_year,feature_200_year,feature_201_year,feature_202_year,feature_203_year,feature_204_year
0,0.172101,1.710832,0.977034,0.288508,-0.131627,-0.493720,-0.006976,-0.004837,-0.005469,-0.005201,...,-0.706066,1.020617,0.789885,0.448605,0.968201,0.435674,0.535790,0.331118,0.531924,-0.824354
1,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,-0.493720,-0.006976,-0.004841,-0.005731,-0.005198,...,1.527059,0.186551,0.387073,0.448605,0.968201,-0.038880,0.102017,0.331118,0.531924,1.160861
2,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,-0.493720,-0.006976,-0.004835,-0.005732,-0.005198,...,1.527059,0.186551,0.789885,0.448605,0.471368,-0.038880,0.102017,0.331118,0.531924,1.160861
3,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,-0.493720,-0.006976,-0.004838,-0.005729,-0.005198,...,-0.706066,-0.230482,0.387073,-0.833280,-0.522300,-0.513433,-0.114869,-1.106292,-1.198660,0.877258
4,0.166140,0.944809,0.584561,-0.079321,-0.051061,-0.139757,-0.006976,-0.004830,-0.005732,-0.005198,...,-0.706066,-0.647515,0.789885,0.448605,-1.019134,0.435674,-0.331756,0.331118,-1.775521,0.026452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,-0.325406,-0.006976,-0.004837,-0.005731,-0.005196,...,-0.706066,-0.230482,-0.418550,-0.833280,-0.522300,-0.513433,-0.114869,1.049822,-1.198660,0.026452
69996,0.309054,1.314150,0.014394,-0.079321,4.153285,0.308755,-0.006976,-0.004845,-0.005727,-0.005198,...,-0.706066,0.603584,0.387073,0.448605,0.471368,0.435674,0.318904,0.331118,0.531924,-0.824354
69997,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,0.255107,-0.006976,-0.004849,-0.005732,-0.005198,...,0.410496,0.186551,0.387073,-0.192338,-0.025466,0.435674,0.102017,0.331118,-0.621799,0.593656
69998,-0.386738,0.884019,0.586300,0.269884,-0.131627,-0.493720,-0.006976,-0.004804,-0.005729,-0.005200,...,-0.706066,1.020617,1.192696,0.448605,0.968201,0.435674,0.535790,0.331118,0.531924,-0.824354


# Feature Engineering - Categorical Features

In [39]:
Categorical_Features = list(data_train.select_dtypes(include = ['object']).columns)

ft = []
num_cat = []
for item in Categorical_Features:
    ft.append(item)
    num_cat.append(data_train[item].nunique())

num_cat = np.array(num_cat).reshape(-1, 1)
num_catf = pd.DataFrame(num_cat, index = ft, columns = ['#'])
num_catf.sort_values(by = '#', ascending = False)

,#
feature_18,15858
feature_133,7126
feature_20,5841
feature_22,2339
feature_21,2327
...,...
feature_187,1
feature_188,1
feature_190,1
feature_248,1


To see the # of categories for each categorical cols.
=> Some just have 1 and some have very large(upto 15858)

In [40]:
one_cat = list(num_catf.loc[num_catf['#'] == 1].index)

#Drop those only have one categorical value
for item in combine:
    item.drop(labels = one_cat, axis =1, inplace = True)
    
print(len(one_cat))
data_train.shape, data_test.shape

6


((70000, 175), (30000, 174))

In [41]:
#Categorical features name
Categorical_Features = list(data_train.select_dtypes(include = ['object']).columns)

#Replacing those categorical values just appearing once with 'other'  
for item in combine:
    for col in Categorical_Features:
        #get a list of categories which contain one value
        tmp = list(item[col].value_counts()[item[col].value_counts() < 2].index)
        item[col] = item[col].apply(lambda x:'other' if x in tmp else x)

ft = []
num_cat = []
for item in Categorical_Features:
    ft.append(item)
    num_cat.append(data_train[item].nunique())

num_cat = np.array(num_cat).reshape(-1, 1)
num_catf = pd.DataFrame(num_cat, index = ft, columns = ['#_cat'])
num_catf.sort_values(by = '#_cat', ascending = False)

,#_cat
feature_18,8678
feature_20,5347
feature_133,2211
feature_19,174
feature_54,174
...,...
feature_1,2
feature_242,2
feature_250,2
feature_270,2


In [42]:
# Import the chi-square 
#p-value level = 0.05
#Null Hypothesis (H0): There is no relationship between feature and target
from scipy.stats import chi2_contingency

chi2_check = []
for i in Categorical_Features:
    if chi2_contingency(pd.crosstab(data_train['target'], data_train[i]))[1] < 0.05:
        chi2_check.append('Reject Null Hypothesis')
    else:
        chi2_check.append('Fail to Reject Null Hypothesis')

res = pd.DataFrame(data = chi2_check, index = Categorical_Features)
res.columns = ['Hypothesis']
res[res['Hypothesis'] == 'Fail to Reject Null Hypothesis']

,Hypothesis
feature_0,Fail to Reject Null Hypothesis
feature_1,Fail to Reject Null Hypothesis
feature_22,Fail to Reject Null Hypothesis
feature_135,Fail to Reject Null Hypothesis
feature_242,Fail to Reject Null Hypothesis
feature_250,Fail to Reject Null Hypothesis
feature_337,Fail to Reject Null Hypothesis
feature_338,Fail to Reject Null Hypothesis


In [43]:
chi_fail = res[res['Hypothesis'] == 'Fail to Reject Null Hypothesis'].index
for item in combine:
    item.drop(labels = chi_fail, axis = 1, inplace = True)
data_train.shape

(70000, 167)

In [44]:
from sklearn.preprocessing import LabelEncoder

Categorical_Features = list(data_train.select_dtypes(include = ['object']).columns)

for col in Categorical_Features:
    le = LabelEncoder()
    #fit encoder on train/test at the same time
    le.fit(pd.concat([data_train[col],data_test[col]], ignore_index=True))
    #transform 
    data_train[col] = le.transform(data_train[col])
    data_test[col] = le.transform(data_test[col])

In [45]:
sc_data = pd.concat([st_X, data_train[Categorical_Features]], axis = 1)
sc_data

,feature_5,feature_7,feature_10,feature_13,feature_15,feature_26,feature_36,feature_37,feature_38,feature_39,...,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_336,feature_371
0,0.172101,1.710832,0.977034,0.288508,-0.131627,-0.493720,-0.006976,-0.004837,-0.005469,-0.005201,...,2,48,23,31,13,2,5,4,0,0
1,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,-0.493720,-0.006976,-0.004841,-0.005731,-0.005198,...,0,29,6,24,8,7,3,2,0,0
2,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,-0.493720,-0.006976,-0.004835,-0.005732,-0.005198,...,0,29,6,24,8,7,3,2,0,0
3,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,-0.493720,-0.006976,-0.004838,-0.005729,-0.005198,...,0,7,93,18,5,3,1,0,0,0
4,0.166140,0.944809,0.584561,-0.079321,-0.051061,-0.139757,-0.006976,-0.004830,-0.005732,-0.005198,...,0,18,104,15,3,3,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,-0.325406,-0.006976,-0.004837,-0.005731,-0.005196,...,0,10,96,21,5,3,1,0,0,1
69996,0.309054,1.314150,0.014394,-0.079321,4.153285,0.308755,-0.006976,-0.004845,-0.005727,-0.005198,...,2,48,23,31,13,2,5,4,0,0
69997,-0.207164,-0.673030,0.014394,-0.079321,-0.131627,0.255107,-0.006976,-0.004849,-0.005732,-0.005198,...,0,11,97,22,6,3,1,0,0,1
69998,-0.386738,0.884019,0.586300,0.269884,-0.131627,-0.493720,-0.006976,-0.004804,-0.005729,-0.005200,...,2,48,23,31,13,2,5,4,0,1


# Model Training 

In [46]:
from sklearn.model_selection import train_test_split
Y = data_train['target']
data_train.drop(['target', 'id'], axis = 1, inplace = True)

X_train, X_test, y_train, y_test = train_test_split(data_train, Y, test_size = 0.3, random_state = 1, stratify = Y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((49000, 165), (21000, 165), (49000,), (21000,))

In [47]:
y_train.mean(), y_train.sum(), y_test.mean(), y_test.sum()

(0.29818367346938773, 14611, 0.2981904761904762, 6262)

To see if the ratio of target valuable is similar in the training and testing sets => checked 

In [48]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score 

rf = RandomForestClassifier(n_estimators = 100)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(rf.score(X_train, y_train), '\n')
print(confusion_matrix(y_test, y_pred), '\n'),
print("Accuracy: ", accuracy_score(y_test, y_pred), '\n'),
print("Precision: ", precision_score(y_test, y_pred), '\n'),
print("Recall: ", recall_score(y_test, y_pred), '\n'),
print("F1:", f1_score(y_test, y_pred), '\n')
print(rf.score(X_test, y_test))
print(roc_auc_score(y_test, y_pred))

0.9999795918367347 

[[13487  1251]
 [ 3911  2351]] 

Accuracy:  0.7541904761904762 

Precision:  0.6526929483620211 

Recall:  0.3754391568189077 

F1: 0.4766828872668289 

0.7541904761904762
0.6452782702265254


In [49]:
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(data_train, Y)
test_predictions = rf.predict(data_test.drop(['id'], axis = 1))
sample = pd.read_csv('../input/netflix-appetency/sample_submission.csv')
sample['target'] = test_predictions
sample.to_csv('submission.csv',index = False)
sample.head(10)

,id,target
0,5,0
1,7,0
2,9,0
3,11,0
4,14,0
5,15,0
6,19,0
7,21,0
8,25,0
9,29,0


In [50]:
sc_X_train, sc_X_test, sc_y_train, sc_y_test = train_test_split(sc_data, Y, test_size = 0.3, random_state = 1, stratify = Y)

In [51]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(sc_X_train, sc_y_train)
sc_y_pred = knn.predict(sc_X_test)

print(knn.score(sc_X_train, sc_y_train), '\n'),
print(confusion_matrix(sc_y_test, sc_y_pred), '\n'),
print("Accuracy: ", accuracy_score(sc_y_test, sc_y_pred), '\n'),
print("Precision: ", precision_score(sc_y_test, sc_y_pred), '\n'),
print("Recall: ", recall_score(sc_y_test, sc_y_pred), '\n'),
print("F1:", f1_score(sc_y_test, sc_y_pred), '\n')
print(roc_auc_score(sc_y_test, sc_y_pred))

0.797795918367347 

[[11553  3185]
 [ 4555  1707]] 

Accuracy:  0.6314285714285715 

Precision:  0.34893704006541293 

Recall:  0.2725966145001597 

F1: 0.30607853684776765 

0.5282442972080117
